In [1333]:
# Install packages not included with Colab
#!pip install python_dotenv
#!pip install alpaca_trade_api
#!pip install backtrader
#!pip install hvplot

# Import Packages
#from alpaca_trade_api import TimeFrameUnit
#from alpaca_trade_api.rest import TimeFrame
#import alpaca_trade_api as tradeapi
#from alpaca_trade_api.stream import Stream
#import backtrader as bt

import matplotlib
import pandas as pd
from pandas.core.frame import DataFrame
import numpy as np
import requests
import matplotlib.pyplot as plt
import hvplot.pandas
from pathlib import Path

#from math import floor
#from termcolor import colored as cl

# Import Machine Learning packages
#from sklearn import svm
#from sklearn.preprocessing import StandardScaler
#from pandas.tseries.offsets import DateOffset
#from sklearn.metrics import classification_report

# Plots parameters
#plt.rcParams['figure.figsize'] = (20,10)
#plt.style.use('fivethirtyeight')

In [1334]:
'''
ALPACA_API_KEY = 'PKZ7UQDMAWO5UAULDXCO'
ALPACA_SECRET_KEY = 'ESBhzVmXVPtUjbzdmKx3airlPQ5eBHlbamnNy123'

alpaca_api = tradeapi.REST(ALPACA_API_KEY,
                ALPACA_SECRET_KEY, 
                'https://paper-api.alpaca.markets',
                'v2')
'''

"\nALPACA_API_KEY = 'PKZ7UQDMAWO5UAULDXCO'\nALPACA_SECRET_KEY = 'ESBhzVmXVPtUjbzdmKx3airlPQ5eBHlbamnNy123'\n\nalpaca_api = tradeapi.REST(ALPACA_API_KEY,\n                ALPACA_SECRET_KEY, \n                'https://paper-api.alpaca.markets',\n                'v2')\n"

In [1335]:
spy_bars = pd.read_csv(
    Path("./data/SPY.csv"),
    index_col="Date",
    parse_dates=True,
    infer_datetime_format=True)

vix_bars = pd.read_csv(
    Path("./data/VIX.csv"),
    index_col="Date",
    parse_dates=True,
    infer_datetime_format=True)


btc_bars = pd.read_csv(
    Path("./data/BTC.csv"),
    index_col="Date",
    parse_dates=True,
    infer_datetime_format=True)


In [1336]:
# Define lookback window and percentage:
lookback = 5
percent = 10

# Generate the short and long window simple moving averages (50 and 100 days, respectively)
# # Generate the trading signal 0 or 1, 
# where 1 is the short-window (SMA50) greater than the long-window (SMA100)
# and 0 is when the condition is not met
short_window =15
long_window =30

In [1337]:
# In this step, we are going to calculate the values of Williams %R by following the formula we discussed before.
def get_William_R (high, low, close, lookback):
    high = high.rolling(lookback).max()
    low = low.rolling(lookback).min()
    wr= -100 * (high - close)/(high- low)
    return wr


In [1338]:
def process_df (df,lookback,short_window,long_window):
    df["Current_WR"] = get_William_R(df["High"], df["Low"], df["Close"], lookback)
    df["Previous_WR"]=df["Current_WR"].shift(periods=lookback)

    # Generate the short and long window simple moving averages (short and long days, respectively)
    df["SMA50"] = df["Close"].rolling(window=short_window).mean()
    df["SMA100"] = df["Close"].rolling(window=long_window).mean()
    df.drop(columns=["Adj Close",'Volume'], inplace=True)
    return df



In [1339]:
btc_plot = process_df(btc_bars, lookback, short_window, long_window)
spy_plot = process_df(spy_bars, lookback,short_window,long_window)
vix_plot = process_df(vix_bars, lookback, short_window,long_window)

In [1340]:
btc_plot

,Open,High,Low,Close,Current_WR,Previous_WR,SMA50,SMA100
Date,,,,,,,,
2017-02-26,1144.270020,1167.469971,1130.199951,1165.199951,NaN,NaN,NaN,NaN
2017-02-27,1163.780029,1181.979980,1163.380005,1179.969971,NaN,NaN,NaN,NaN
2017-02-28,1180.719971,1193.250000,1171.819946,1179.969971,NaN,NaN,NaN,NaN
2017-03-01,1180.040039,1222.500000,1179.689941,1222.500000,NaN,NaN,NaN,NaN
2017-03-02,1224.680054,1262.130005,1215.619995,1251.010010,-8.428705,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2022-02-22,37068.769531,38359.855469,36488.933594,38286.027344,-59.526910,-93.466488,41632.005990,40214.921094
2022-02-23,38285.281250,39122.394531,37201.816406,37296.570313,-79.449161,-92.179115,41177.214323,40236.329167
2022-02-24,37278.566406,38968.839844,34459.218750,38332.609375,-31.573659,-90.366254,40776.801823,40282.282682


In [1341]:
# # strategy1
def Strategy_1(df, percent: int):
    #df.dropna(inplace=True)
    # Add a new column
    df["WR_BS"] =0.0
    df["MA_Signal"] =0.0
    df["MA_BS"] =0.0
    df["Buy_Sell"] =0.0 
    df["Entry/Exit_S1"] =0.0
    for index, row in df.iterrows():
        if (row["Previous_WR"] < -percent and row["Current_WR"] > -percent):
            df.loc[index,"WR_BS"] = 1
        elif (row["Previous_WR"] > -(100-percent) and row["Current_WR"] <-(100-percent)):
            df.loc[index,"WR_BS"]  = 0
# Generate the trading signal 0 or 1,
# where 1 is the short-window (SMA50) greater than the long-window (SMA100)
# and 0 is when the condition is not met
    df["MA_Signal"][short_window:] = np.where(df["SMA50"][short_window:] > df["SMA100"][short_window:], 1.0, 0.0)
    df["MA_BS"] = df["MA_Signal"].diff()
    df["Buy_Sell"] = np.where((df["WR_BS"] +  df["MA_Signal"]==2.0),1.0,0.0)   
    # Calculate the points in time when the Signal value changes
    # Identify trade entry (1) and exit (-1) points
    df["Entry/Exit_S1"] = df["Buy_Sell"].diff()


In [1342]:
# strategy2
def Strategy_2(df, percent: int):
    #df.dropna(inplace=True)
    # Add a new column
    df["Buy1_Sell0"] =0.0
    df["MA_Signal"] =0.0
    df["Buy_Sell"] =0.0 
    df["Entry/Exit_S2"] =0.0
    for index, row in df.iterrows():
        if (row["Previous_WR"] < -percent and row["Current_WR"] > -percent):
            df.loc[index,"Buy1_Sell0"] = 1.0
        elif (row["Previous_WR"] > -(100-percent) and row["Current_WR"] <-(100-percent)):
            df.loc[index,"Buy1_Sell0"]  = -1.0
    # Calculate the points in time when the Signal value changes
    # Identify trade entry (1) and exit (-1) points
    df["Entry/Exit_S2"] = df["Buy1_Sell0"].diff()


In [1343]:
Strategy_1(btc_plot, 20)
Strategy_1(spy_plot, 20)
Strategy_1(vix_plot, 20)


In [1344]:
def plot_Strategy_1(df, name:str,list_of_years):
    # Slice the plot data with year from the date: get the year out of date
    #year_df = pd.DataFrame({'Year':list(year)})
    df['Year'] = pd.to_datetime(df.index).to_period("Y")
    df['Year'] = df['Year'].dt.strftime('%Y')
    df = df[df["Year"].isin(list_of_years) ]
    #df = df_display[df_display["Year"]==year ]
    # Visualize exit position relative to close price
    exit = df[df['Entry/Exit_S1'] <0.0][['Close','Year']].hvplot.scatter(
        y ="Close", 
        color='green',
        marker='v',
        size=200,
        legend=False,
        ylabel='Price in $',
        width=1000,
        height=400,
        #groupby ="Year",
    )

    # Visualize entry position relative to close price
    entry = df[df['Entry/Exit_S1'] > 0.0][['Close','Year']].hvplot.scatter(
        y ="Close", 
        color='blue',
        marker='^',
        size=200,
        legend=False,
        ylabel='Price in $',
        width=1000,
        height=400,
        #groupby ="Year",
    )

    df_close = df[['Close','Year']].hvplot(
        y="Close",
        line_color='red',
        ylabel='Price in $',
        width=1000,
        height=400,
        #groupby ="Year",
    )

    plot = exit * entry * df_close
    plot.opts(
        title= name + " BOV Entry and Exit Points with Strategy 1"
    )

    return plot

In [1345]:
#list_of_years =['2017','2018','2019','2020','2021','2022']
list_of_years =['2017','2018']

In [1346]:
BTC_BS_plot = plot_Strategy_1(btc_plot,"BTC",list_of_years)
BTC_BS_plot

:Overlay
   .Scatter.I  :Scatter   [Date]   (Close)
   .Scatter.II :Scatter   [Date]   (Close)
   .Curve.I    :Curve   [Date]   (Close)

In [1347]:
SPY_BS_plot = plot_Strategy_1(spy_plot,"SPY",list_of_years)
SPY_BS_plot

:Overlay
   .Scatter.I  :Scatter   [Date]   (Close)
   .Scatter.II :Scatter   [Date]   (Close)
   .Curve.I    :Curve   [Date]   (Close)

In [1348]:
VIX_BS_plot= plot_Strategy_1(vix_plot,"VIX",list_of_years)
VIX_BS_plot

:Overlay
   .Scatter.I  :Scatter   [Date]   (Close)
   .Scatter.II :Scatter   [Date]   (Close)
   .Curve.I    :Curve   [Date]   (Close)

In [1349]:
Strategy_2(btc_plot, 20)
Strategy_2(spy_plot, 20)
Strategy_2(vix_plot, 20)

In [1350]:
def plot_Strategy_2(df, name:str,list_of_years):
    # Slice the plot data with year from the date: get the year out of date
    #year_df = pd.DataFrame({'Year':list(year)})
    df['Year'] = pd.to_datetime(df.index).to_period("Y")
    df['Year'] = df['Year'].dt.strftime('%Y')
    df = df[df["Year"].isin(list_of_years) ]
    #df = df_display[df_display["Year"]==year ]
    # Visualize exit position relative to close price
    exit = df[df['Entry/Exit_S2'] <0.0][['Close','Year']].hvplot.scatter(
        y ="Close", 
        color='green',
        marker='v',
        size=200,
        legend=False,
        ylabel='Price in $',
        width=1000,
        height=400,
        #groupby ="Year",
    )

    # Visualize entry position relative to close price
    entry = df[df['Entry/Exit_S2'] > 0.0][['Close','Year']].hvplot.scatter(
        y ="Close", 
        color='blue',
        marker='^',
        size=200,
        legend=False,
        ylabel='Price in $',
        width=1000,
        height=400,
        #groupby ="Year",
    )

    df_close = df[['Close','Year']].hvplot(
        y="Close",
        line_color='red',
        ylabel='Price in $',
        width=1000,
        height=400,
        #groupby ="Year",
    )

    plot = exit * entry * df_close
    plot.opts(
        title= name + " BOV Entry and Exit Points with Strategy 2"
    )

    return plot

In [1351]:
BTC_BS_plot_2 = plot_Strategy_2(btc_plot,"BTC",list_of_years)
BTC_BS_plot_2

:Overlay
   .Scatter.I  :Scatter   [Date]   (Close)
   .Scatter.II :Scatter   [Date]   (Close)
   .Curve.I    :Curve   [Date]   (Close)

In [1352]:
SPY_BS_plot_2 = plot_Strategy_2(spy_plot,"VIX",list_of_years)
SPY_BS_plot_2

:Overlay
   .Scatter.I  :Scatter   [Date]   (Close)
   .Scatter.II :Scatter   [Date]   (Close)
   .Curve.I    :Curve   [Date]   (Close)

In [1353]:
VIX_BS_plot_2 = plot_Strategy_2(vix_plot,"VIX",list_of_years)
VIX_BS_plot_2

:Overlay
   .Scatter.I  :Scatter   [Date]   (Close)
   .Scatter.II :Scatter   [Date]   (Close)
   .Curve.I    :Curve   [Date]   (Close)

### Backtest the trading strategy

In [1354]:
# Set initial capital
initial_capital = float(100000)

# Set the share size
share_size = -500

In [1355]:
#Strategy1 backtest
def calculteS1_returns(df): 
    df["Position"] = share_size * np.where(df["Entry/Exit_S1"] > 0.0, 1, (np.where(df["Entry/Exit_S1"] <0.0, -1,0))) 

    # Multiply share price by entry/exit positions and get the cumulatively sum
    df["Portfolio_Holdings"] = df["Close"]* df["Position"].cumsum()

    # Subtract the initial capital by the portfolio holdings to get the amount of liquid cash in the portfolio
    df["Portfolio_Cash"] = (
        initial_capital - (df["Close"] * df["Position"]).cumsum()
    )

    # Get the total portfolio value by adding the cash amount by the portfolio holdings (or investments)
    df["Portfolio_Total"] = df["Portfolio_Cash"] + df["Portfolio_Holdings"]

    # Calculate the portfolio daily returns
    df["Portfolio_Daily_Returns"] = df["Portfolio_Total"].pct_change()

    # Calculate the cumulative returns
    df["Portfolio Cumulative Returns"] = (1 + df["Portfolio_Daily_Returns"]).cumprod() - 1



In [1356]:
calculteS1_returns(btc_plot)
calculteS1_returns(spy_plot)
calculteS1_returns(vix_plot)

In [1357]:
def display_returns(df, Strategy_name, name:str):
    plot = df["Portfolio Cumulative Returns"].hvplot(title = name +" Portfolio Cumulative Returns of " + Strategy_name)
    return plot 

In [1358]:
# import panel as pn
# from panel.interact import interact


# # Put parallel plots in a single row
# Column1 = pn.Column(
#     display_returns(spy_plot,'Strategy 1', 'SPY'),
#     display_returns(vix_plot,'Strategy 1', 'VIX'),
#     display_returns(btc_plot,'Strategy 1', 'BTC'), 
# )
# Column1


In [1359]:
BTC_return_plot = display_returns(btc_plot,'Strategy 1', 'BTC')
BTC_return_plot

:Curve   [Date]   (Portfolio Cumulative Returns)

In [1360]:
SPY_return_plot = display_returns(spy_plot,'Strategy 1', 'SPY')
SPY_return_plot

:Curve   [Date]   (Portfolio Cumulative Returns)

In [1361]:
VIX_return_plot = display_returns(vix_plot,'Strategy 1', 'VIX')
VIX_return_plot

:Curve   [Date]   (Portfolio Cumulative Returns)

In [1362]:
#Strategy1 backtest
def calculteS2_returns(df): 
    df["Position"] = share_size * np.where(df["Entry/Exit_S2"] > 0.0, 1, (np.where(df["Entry/Exit_S2"] <0.0, -1,0))) 

    # Multiply share price by entry/exit positions and get the cumulatively sum
    df["Portfolio_Holdings"] = df["Close"]* df["Position"].cumsum()

    # Subtract the initial capital by the portfolio holdings to get the amount of liquid cash in the portfolio
    df["Portfolio_Cash"] = (
        initial_capital - (df["Close"] * df["Position"]).cumsum()
    )

    # Get the total portfolio value by adding the cash amount by the portfolio holdings (or investments)
    df["Portfolio_Total"] = df["Portfolio_Cash"] + df["Portfolio_Holdings"]

    # Calculate the portfolio daily returns
    df["Portfolio_Daily_Returns"] = df["Portfolio_Total"].pct_change()

    # Calculate the cumulative returns
    df["Portfolio Cumulative Returns"] = (1 + df["Portfolio_Daily_Returns"]).cumprod() - 1



In [1363]:
calculteS2_returns(btc_plot)
calculteS2_returns(spy_plot)
calculteS2_returns(vix_plot)

In [1364]:
BTC_return_plot_2 = display_returns(btc_plot,'Strategy 2', 'BTC')
BTC_return_plot_2

:Curve   [Date]   (Portfolio Cumulative Returns)

In [1365]:
SPY_return_plot_2 = display_returns(spy_plot,'Strategy 2', 'SPY')
SPY_return_plot_2

:Curve   [Date]   (Portfolio Cumulative Returns)

In [1366]:
VIX_return_plot_2 = display_returns(vix_plot,'Strategy 2', 'VIX')
VIX_return_plot_2

:Curve   [Date]   (Portfolio Cumulative Returns)